In [ ]:
from autogen_core import CancellationToken, TypeSubscription, TopicId, AgentId, type_subscription
from autogen_agentchat.messages import TextMessage, Image, UserMessage
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import DiGraph, DiGraphBuilder, GraphFlow
from autogen_core.code_executor import CodeBlock, CodeExecutor
from autogen_ext.code_executors.local import LocalCommandLineCodeExecutor
from autogen_ext.code_executors.docker import DockerCommandLineCodeExecutor
from autogen_ext.tools.code_execution import PythonCodeExecutionTool
from autogen_ext.models.openai import OpenAIChatCompletionClient

from pathlib import Path

working_dir = Path("coding")
model_client = OpenAIChatCompletionClient(model="gpt-4o")

inst_sub = TypeSubscription(topic_type="instruction", agent_type="assistant_agent")
exec_sub = TypeSubscription(topic_type="execution", agent_type="python_agent")
res_sub = TypeSubscription(topic_type="result", agent_type="assistant_agent")

In [27]:
from autogen_core import SingleThreadedAgentRuntime
from autogen_core.models import AssistantMessage
runtime = SingleThreadedAgentRuntime()

# Register subscriptions
await runtime.add_subscription(inst_sub)
await runtime.add_subscription(exec_sub)
await runtime.add_subscription(res_sub)

# Define Agents
assistant = await AssistantAgent.register(
    runtime,
    "assistant_agent",
    lambda: AssistantAgent(model_client=model_client, system_message="You are the assisting LLM.")
)

# Python execution agent (simple RoutedAgent)
# Uses LocalCommandLineCodeExecutor to execute code blocks
from autogen_core import RoutedAgent, message_handler, MessageContext
from autogen_core.models import LLMMessage

class PythonAgent(RoutedAgent):
    @message_handler
    async def run_code(self, message: AssistantMessage, ctx: MessageContext):
        code = message.content
        with LocalCommandLineCodeExecutor(work_dir=working_dir) as executor:
            result = await executor.execute_code_blocks(
                code_blocks=[CodeBlock(language="python", code=code)],
                cancellation_token=CancellationToken(),
            )
        output = result.output or ""
        await self.publish_message(
            AssistantMessage(content=output),
            topic_id=TopicId(type="result", source=ctx.topic_id.source)
        )

python_agent = await PythonAgent.register(
    runtime, "python_agent", lambda: PythonAgent("Python Executor")
)

# Start runtime
runtime.start()

# Simulate user question
user_topic = "session1"
await runtime.publish_message(
    UserMessage(content="Make a plot of x vs y"),
    topic_id=TopicId(type="instruction", source=user_topic),
)

await runtime.stop_when_idle()

TypeError: ABCMeta.register() takes 2 positional arguments but 4 were given

In [ ]:
assistant_agent = AssistantAgent(
    name="assistant_agent",
    description="An assistant agent that analyses user queries and provides helpful responses.",
    system_message="""
    You are a helpful assistant you analyze user queries and provide helpful responses.

    if the user asks a question which requires code execution, you should delegate the task to the code agent by saying clear instructions about what code to run and what the expected output is.
    """,
    model_client=model_client
)
code_agent = AssistantAgent(
    name="code_agent",
    description="A code generation agent that generates code snippets and returns the output.",
    system_message="""
    You are a code generation agent you generate code snippets and use the code execution tool to run them.
    """,
    model_client=model_client,
)

In [20]:
builder = DiGraphBuilder()
builder.add_node(assistant_agent).add_node(code_agent)
builder.add_edge(assistant_agent, code_agent)

graph = builder.build()

flow = GraphFlow(participants=[assistant_agent, code_agent], graph=graph)

In [21]:
flow._participant_descriptions

['An assistant agent that analyses user queries and provides helpful responses.',
 'A code generation agent that generates code snippets in python and returns the output.']

In [22]:
# Use `asyncio.run(...)` and wrap the below in a async function when running in a script.
stream = flow.run_stream(task="Give me a code snippet that prints 'Hello, World!'")
async for event in stream:  # type: ignore
    print(event)
# Use Console(flow.run_stream(...)) for better formatting in console.


id='75029450-5439-401c-8099-8c6c0d90cc06' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 8, 10, 18, 45, 12, 134068, tzinfo=datetime.timezone.utc) content="Give me a code snippet that prints 'Hello, World!'" type='TextMessage'
id='a6f8320a-2288-4319-a1fc-b737f06266d0' source='assistant_agent' models_usage=RequestUsage(prompt_tokens=78, completion_tokens=52) metadata={} created_at=datetime.datetime(2025, 8, 10, 18, 45, 13, 967532, tzinfo=datetime.timezone.utc) content="Sure! Here's a simple code snippet in Python that prints 'Hello, World!':\n\n```python\nprint('Hello, World!')\n```\n\nYou can run this code in any Python environment, and it will display `Hello, World!` as the output." type='TextMessage'


Error processing publish message for code_agent_1dd06c88-c353-4c1a-a4c6-42079b6daa48/1dd06c88-c353-4c1a-a4c6-42079b6daa48
Traceback (most recent call last):
  File "d:\github\autogen-examples\.venv\Lib\site-packages\autogen_core\_single_threaded_agent_runtime.py", line 606, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\github\autogen-examples\.venv\Lib\site-packages\autogen_core\_base_agent.py", line 119, in on_message
    return await self.on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\github\autogen-examples\.venv\Lib\site-packages\autogen_agentchat\teams\_group_chat\_sequential_routed_agent.py", line 67, in on_message_impl
    return await super().on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\github\autogen-examples\.venv\Lib\site-packages\autogen_core\_routed_agent.py", line 485, in on_message_impl
    return await h(self, message, ctx)
 

id='f8d2ae6f-d002-4641-82c0-d9d2c3cdee6d' source='code_agent' models_usage=RequestUsage(prompt_tokens=153, completion_tokens=20) metadata={} created_at=datetime.datetime(2025, 8, 10, 18, 45, 15, 434743, tzinfo=datetime.timezone.utc) content=[FunctionCall(id='call_InVgLelK8tnERbD4tDnyQagH', arguments='{"code":"print(\'Hello, World!\')"}', name='CodeExecutor')] type='ToolCallRequestEvent'
id='720b1834-2b4f-4bd5-b571-45094b93059a' source='code_agent' models_usage=None metadata={} created_at=datetime.datetime(2025, 8, 10, 18, 45, 15, 435742, tzinfo=datetime.timezone.utc) content=[FunctionExecutionResult(content="'coroutine' object has no attribute 'execute_code_blocks'", name='CodeExecutor', call_id='call_InVgLelK8tnERbD4tDnyQagH', is_error=True)] type='ToolCallExecutionEvent'


RuntimeError: TypeError: Cannot instantiate typing.Union
Traceback:
Traceback (most recent call last):

  File "d:\github\autogen-examples\.venv\Lib\site-packages\autogen_agentchat\teams\_group_chat\_base_group_chat.py", line 502, in stop_runtime
    await self._runtime.stop_when_idle()

  File "d:\github\autogen-examples\.venv\Lib\site-packages\autogen_core\_single_threaded_agent_runtime.py", line 851, in stop_when_idle
    await self._run_context.stop_when_idle()

  File "d:\github\autogen-examples\.venv\Lib\site-packages\autogen_core\_single_threaded_agent_runtime.py", line 121, in stop_when_idle
    await self._run_task

  File "d:\github\autogen-examples\.venv\Lib\site-packages\autogen_core\_single_threaded_agent_runtime.py", line 110, in _run
    await self._runtime._process_next()  # type: ignore
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "d:\github\autogen-examples\.venv\Lib\site-packages\autogen_core\_single_threaded_agent_runtime.py", line 678, in _process_next
    raise

d:\github\autogen-examples\.venv\Lib\site-packages\pygments\regexopt.py:77: RuntimeWarning: coroutine 'DockerCommandLineCodeExecutor.start' was never awaited
  '|'.join(regex_opt_inner(list(group[1]), '')
